In [27]:
import torch
from torch import nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

import os
import numpy as np
import pandas as pd
import wandb

import plotly.express as px

# Pytorch implementation

In [28]:
config = {
    "lr": { 'values': [1e-2] },
    "epochs": { 'values': [50] },
    "batch_size": { 'values': [64] },
    "hidden_layer1": { 'values': [32] },
    "hidden_layer2": { 'values': [16] },
    'dropout': { 'values': [0.3] },
    'gamma': { 'min': 0.6, 'max': 0.99 },
    'beta1': { 'min': 0.8, 'max': 0.99 },
    'beta2': { 'min': 0.8, 'max': 0.99 },
}

In [29]:
sweep_configuration = {
    'method': 'random',
    'name': 'sweep',
    'metric': {'goal': 'minimize', 'name': 'val_loss'},
}

sweep_configuration['parameters'] = config

In [ ]:
wandb.login()
sweep_id = wandb.sweep(sweep=sweep_configuration, project='assignment-1-sweep')

In [31]:
# device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
device = 'cpu'
print(f"Using {device} device")

Using cpu device


## Loading the dataset

In [32]:
class TitanicDataset(Dataset):
    def __init__(self, data_dir):
        self.labels = pd.read_csv(os.path.join(data_dir, 'labels.csv')).to_numpy(dtype='float32')
        self.data = pd.read_csv(os.path.join(data_dir, 'data.csv')).to_numpy(dtype='float32')

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        data = torch.from_numpy(self.data[idx])
        label = torch.from_numpy(self.labels[idx])
        
        return data, label

In [33]:
train_data = TitanicDataset(data_dir='data/train/')
val_data = TitanicDataset(data_dir='data/val/')

## Defining the model

In [34]:
class NeuralNetwork(nn.Module):
    def __init__(self, hidden_layer1, hidden_layer2, dropout):
        super().__init__()
        self.linear = nn.Sequential(
            nn.Linear(6, hidden_layer1),
            # nn.Dropout(config['dropouts'][0]),
            nn.ReLU(),
            nn.Linear(hidden_layer1, hidden_layer2),
            nn.Dropout(dropout),
            nn.ReLU(),
            nn.Linear(hidden_layer2, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        out = self.linear(x)
        return out

## Training

In [35]:
def calculate_acc(y_pred, y) -> float:
    y_pred = (y_pred>0.5)
    
    return ((y == y_pred).sum().item())/len(y)

In [36]:
def train(dataloader, model, loss_fn, optimizer, scheduler, epoch):
    model.train()
    avg_loss, avg_acc = 0, 0
    for x, y in dataloader:
        x, y = x.to(device), y.to(device)

        # Get prediction and compute loss
        y_pred = model(x)
        loss = loss_fn(y_pred, y)
        avg_loss += loss.item()

        # Update parameters
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        avg_acc += calculate_acc(y_pred, y)

    # Change lr based on the provided scheduler
    if epoch > 9:
        scheduler.step()
    
    avg_loss /= len(dataloader)
    avg_acc /= len(dataloader)
        
    return (avg_loss, avg_acc)

In [37]:
def val(dataloader, model, loss_fn, epoch):    
    model.eval()
    avg_loss, avg_acc = 0, 0
    with torch.no_grad():
        for x, y in dataloader:
            x, y = x.to(device), y.to(device)
            y_pred = model(x)
            avg_loss += loss_fn(y_pred, y).item()
            avg_acc += calculate_acc(y_pred, y)
    
    avg_loss /= len(dataloader)
    avg_acc /= len(dataloader)
    
    return (avg_loss, avg_acc)

In [38]:
def fn(config=None):
    with wandb.init(config=config):
        config = wandb.config

        train_dataloader = DataLoader(train_data, batch_size=config['batch_size'], shuffle=True)
        val_dataloader = DataLoader(val_data, batch_size=config['batch_size'], shuffle=False)
        
        train_history = {'loss': [], 'acc': []}
        val_history = {'loss': [], 'acc': []}
        
        model = NeuralNetwork(config['hidden_layer1'], config['hidden_layer2'], config['dropout']).to(device)
        
        loss_fn = nn.BCELoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=config['lr'], betas=[config['beta1'], config['beta2']])
        scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=config['gamma'])
        
        epochs = config['epochs']        
        for epoch in range(epochs):
            train_loss, train_acc = train(train_dataloader, model, loss_fn, optimizer, scheduler, epoch)
            val_loss, val_acc = val(val_dataloader, model, loss_fn, epoch)

            train_history['loss'].append(train_loss)
            train_history['acc'].append(train_acc)

            val_history['loss'].append(val_loss)
            val_history['acc'].append(val_acc)
            
            wandb.log({'epoch': epoch, 'loss': train_loss, 'accuracy': train_acc, 'val_loss':val_loss, 'val_accuracy': val_acc, 'lr': optimizer.param_groups[0]["lr"]})

In [ ]:
wandb.agent(sweep_id, fn, count=100)